Install and import library modules. Download English stopwords and WordNet. Set logging to INFO

In [1]:
import sys
!{sys.executable} -m pip install gensim
!{sys.executable} -m pip install pyLDAvis
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install matplotlib
!{sys.executable} -m pip install nltk

import re, gensim, pyLDAvis, logging, datetime, multiprocessing, os, pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from math import floor, ceil
from nltk.corpus import stopwords
from nltk import download
from nltk.stem import WordNetLemmatizer

download('stopwords')
download('wordnet')

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/joostgadellaa/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/joostgadellaa/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Read the data as pandas dataframe, remove items with missing abstracts, and fill other missing data with empty strings

In [2]:
df = pd.read_csv('data/data_merged_and_filtered.csv')
df = df[df['abstract'].notna()]
df.fillna("", inplace=True)

Combine the text fiels into one

In [3]:
df["data"] = df["abstract"] + " " + df["keywords"] + " " + df["title"]

Remove copyright notices from the text

In [4]:
copyright_re = "(©|&#x00A9;|Copyright)(.*?(?=B\\.V\\.|Ltd|\\.))(B\\.V\\.|Ltd|\\.)"
rights = "All rights reserved."
df.replace(copyright_re, '', regex=True, inplace=True)
df.replace(rights, '', regex=False, inplace=True)

Tokenize, convert to lowercase, remove punctuation, remove stop-words, lemmatize

In [5]:
def preprocess(text):
  result = [word for word in gensim.utils.simple_preprocess(text, deacc=True) if word not in stop_words]
  wordnet_lemmatizer = WordNetLemmatizer()
  result = [wordnet_lemmatizer.lemmatize(word) for word in result]
  return result

stop_words = stopwords.words('english')
df["data_preprocessed"] = df["data"].map(preprocess)

Generate the Dictionairy and perform relative pruning

In [6]:
dictionary = gensim.corpora.Dictionary(df["data_preprocessed"])
dictionary.filter_extremes(no_below=(floor(0.005 * len(df))), no_above=0.99)

2021-07-19 11:44:21,344 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2021-07-19 11:44:23,236 : INFO : adding document #10000 to Dictionary(27074 unique tokens: ['ability', 'acceptable', 'additionally', 'analysis', 'anti']...)
2021-07-19 11:44:25,119 : INFO : adding document #20000 to Dictionary(38089 unique tokens: ['ability', 'acceptable', 'additionally', 'analysis', 'anti']...)
2021-07-19 11:44:27,025 : INFO : adding document #30000 to Dictionary(46717 unique tokens: ['ability', 'acceptable', 'additionally', 'analysis', 'anti']...)
2021-07-19 11:44:28,950 : INFO : adding document #40000 to Dictionary(53696 unique tokens: ['ability', 'acceptable', 'additionally', 'analysis', 'anti']...)
2021-07-19 11:44:30,737 : INFO : adding document #50000 to Dictionary(59648 unique tokens: ['ability', 'acceptable', 'additionally', 'analysis', 'anti']...)
2021-07-19 11:44:32,409 : INFO : adding document #60000 to Dictionary(64997 unique tokens: ['ability', 'acceptable', 'additional

Make the bag of words

In [7]:
corpus = [dictionary.doc2bow(doc) for doc in df["data_preprocessed"]]

Generate all models for evaluation

In [8]:
k = [25, 30, 36, 40, 42, 49] #[1, 4, 9, 16, 25, 36, 49, 64, 81, 100, 121, 144, 169, 196, 225, 256, 289, 324, 361] later added [30, 40, 42]
random_states = [24, 27, 1024, 1027] #[24, 1024] later added [27, 1027]

texts = df["data_preprocessed"]
chunksize = ceil(len(texts))
passes = 20
iterations = 100
eval_every = None
dir_prior = 'auto'

for random_state in random_states:
    for num_topics in k:
        
        name = "rs{}k{}".format(random_state, num_topics)
        
        
        if 'model_' + name in os.listdir('models'):
            print(datetime.datetime.now().strftime('%H:%M:%S') + ' Model ' + name + ' already in directory. Skipped')
        
        else:
            print(datetime.datetime.now().strftime('%H:%M:%S') + " Building model " + name)
            lda_model = gensim.models.LdaModel(corpus, num_topics=num_topics, id2word=dictionary, alpha=dir_prior, eta=dir_prior, random_state=random_state, chunksize=chunksize, passes=passes, iterations=iterations)

            lda_model.save(os.path.join('models', 'model_' + name))

11:44:45 Model rs24k25 already in directory. Skipped
11:44:45 Model rs24k30 already in directory. Skipped
11:44:45 Model rs24k36 already in directory. Skipped
11:44:45 Model rs24k40 already in directory. Skipped
11:44:45 Model rs24k42 already in directory. Skipped
11:44:45 Model rs24k49 already in directory. Skipped
11:44:45 Model rs27k25 already in directory. Skipped
11:44:45 Model rs27k30 already in directory. Skipped
11:44:45 Model rs27k36 already in directory. Skipped
11:44:45 Model rs27k40 already in directory. Skipped
11:44:45 Model rs27k42 already in directory. Skipped
11:44:45 Model rs27k49 already in directory. Skipped
11:44:45 Model rs1024k25 already in directory. Skipped
11:44:45 Model rs1024k30 already in directory. Skipped
11:44:45 Model rs1024k36 already in directory. Skipped
11:44:45 Model rs1024k40 already in directory. Skipped
11:44:45 Model rs1024k42 already in directory. Skipped
11:44:45 Model rs1024k49 already in directory. Skipped
11:44:45 Model rs1027k25 already i

Evaluate all models present in the current directory

In [ ]:
models_to_evaluate = []

for entry in os.scandir('models'):
    if entry.path.endswith('.state'):
        models_to_evaluate.append(entry.path[:-6])
        
models_to_evaluate.sort()        
        
for path in models_to_evaluate:
    print(datetime.datetime.now().strftime('%H:%M:%S') + ' Starting evaluation of ' + path)
    model = gensim.models.LdaModel.load(path)
    coherence_model_lda = gensim.models.coherencemodel.CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
    coherence = coherence_model_lda.get_coherence()
        
    with open(os.path.join('models', 'scores.txt'), "a") as f:
        f.write("\"" + str(os.path.split(path)[1]) + "\"" ' : ' + str(coherence) + ',\n')   

2021-07-19 11:44:45,269 : INFO : loading LdaModel object from models/model_rs1024k1
2021-07-19 11:44:45,276 : INFO : loading expElogbeta from models/model_rs1024k1.expElogbeta.npy with mmap=None
2021-07-19 11:44:45,279 : INFO : setting ignored attribute id2word to None
2021-07-19 11:44:45,281 : INFO : setting ignored attribute state to None
2021-07-19 11:44:45,288 : INFO : setting ignored attribute dispatcher to None
2021-07-19 11:44:45,304 : INFO : LdaModel lifecycle event {'fname': 'models/model_rs1024k1', 'datetime': '2021-07-19T11:44:45.304116', 'gensim': '4.0.1', 'python': '3.9.6 (v3.9.6:db3ff76da1, Jun 28 2021, 11:49:53) \n[Clang 6.0 (clang-600.0.57)]', 'platform': 'macOS-10.15.7-x86_64-i386-64bit', 'event': 'loaded'}
2021-07-19 11:44:45,313 : INFO : loading LdaState object from models/model_rs1024k1.state
2021-07-19 11:44:45,352 : INFO : LdaState lifecycle event {'fname': 'models/model_rs1024k1.state', 'datetime': '2021-07-19T11:44:45.352676', 'gensim': '4.0.1', 'python': '3.9.6

11:44:45Starting evaluation of model models/model_rs1024k1


2021-07-19 11:44:45,837 : INFO : 1 batches submitted to accumulate stats from 64 documents (1275 virtual)
2021-07-19 11:44:45,854 : INFO : 2 batches submitted to accumulate stats from 128 documents (2284 virtual)
2021-07-19 11:44:45,872 : INFO : 3 batches submitted to accumulate stats from 192 documents (2853 virtual)
2021-07-19 11:44:47,254 : INFO : 5 batches submitted to accumulate stats from 320 documents (5536 virtual)
2021-07-19 11:44:47,266 : INFO : 6 batches submitted to accumulate stats from 384 documents (7420 virtual)
2021-07-19 11:44:47,359 : INFO : 7 batches submitted to accumulate stats from 448 documents (9671 virtual)
2021-07-19 11:44:47,382 : INFO : 8 batches submitted to accumulate stats from 512 documents (12615 virtual)
2021-07-19 11:44:47,412 : INFO : 9 batches submitted to accumulate stats from 576 documents (14328 virtual)
2021-07-19 11:44:47,448 : INFO : 10 batches submitted to accumulate stats from 640 documents (16710 virtual)
2021-07-19 11:44:47,571 : INFO : 1

2021-07-19 11:44:51,306 : INFO : 77 batches submitted to accumulate stats from 4928 documents (125831 virtual)
2021-07-19 11:44:51,419 : INFO : 78 batches submitted to accumulate stats from 4992 documents (127422 virtual)
2021-07-19 11:44:51,429 : INFO : 79 batches submitted to accumulate stats from 5056 documents (129413 virtual)
2021-07-19 11:44:51,464 : INFO : 80 batches submitted to accumulate stats from 5120 documents (131502 virtual)
2021-07-19 11:44:51,554 : INFO : 81 batches submitted to accumulate stats from 5184 documents (133148 virtual)
2021-07-19 11:44:51,581 : INFO : 82 batches submitted to accumulate stats from 5248 documents (134804 virtual)
2021-07-19 11:44:51,631 : INFO : 83 batches submitted to accumulate stats from 5312 documents (135753 virtual)
2021-07-19 11:44:51,712 : INFO : 84 batches submitted to accumulate stats from 5376 documents (136836 virtual)
2021-07-19 11:44:51,761 : INFO : 85 batches submitted to accumulate stats from 5440 documents (137831 virtual)
2

2021-07-19 11:44:55,211 : INFO : 154 batches submitted to accumulate stats from 9856 documents (235742 virtual)
2021-07-19 11:44:55,286 : INFO : 155 batches submitted to accumulate stats from 9920 documents (238031 virtual)
2021-07-19 11:44:55,339 : INFO : 156 batches submitted to accumulate stats from 9984 documents (239112 virtual)
2021-07-19 11:44:55,351 : INFO : 157 batches submitted to accumulate stats from 10048 documents (240020 virtual)
2021-07-19 11:44:55,397 : INFO : 158 batches submitted to accumulate stats from 10112 documents (241148 virtual)
2021-07-19 11:44:55,436 : INFO : 159 batches submitted to accumulate stats from 10176 documents (242364 virtual)
2021-07-19 11:44:55,489 : INFO : 160 batches submitted to accumulate stats from 10240 documents (243828 virtual)
2021-07-19 11:44:55,560 : INFO : 161 batches submitted to accumulate stats from 10304 documents (246652 virtual)
2021-07-19 11:44:55,593 : INFO : 162 batches submitted to accumulate stats from 10368 documents (24

...

2021-07-19 12:39:47,307 : INFO : 458 batches submitted to accumulate stats from 29312 documents (576634 virtual)
2021-07-19 12:39:48,780 : INFO : 459 batches submitted to accumulate stats from 29376 documents (576656 virtual)
2021-07-19 12:39:49,788 : INFO : 460 batches submitted to accumulate stats from 29440 documents (577583 virtual)
2021-07-19 12:39:50,202 : INFO : 461 batches submitted to accumulate stats from 29504 documents (579012 virtual)
2021-07-19 12:39:51,845 : INFO : 462 batches submitted to accumulate stats from 29568 documents (580541 virtual)
2021-07-19 12:39:52,662 : INFO : 463 batches submitted to accumulate stats from 29632 documents (581301 virtual)
2021-07-19 12:39:52,811 : INFO : 464 batches submitted to accumulate stats from 29696 documents (582062 virtual)
2021-07-19 12:39:54,228 : INFO : 465 batches submitted to accumulate stats from 29760 documents (582436 virtual)
2021-07-19 12:39:55,267 : INFO : 466 batches submitted to accumulate stats from 29824 documents 

2021-07-19 12:41:07,627 : INFO : 541 batches submitted to accumulate stats from 34624 documents (642386 virtual)
2021-07-19 12:41:08,946 : INFO : 542 batches submitted to accumulate stats from 34688 documents (643343 virtual)
2021-07-19 12:41:10,643 : INFO : 543 batches submitted to accumulate stats from 34752 documents (643481 virtual)
2021-07-19 12:41:10,916 : INFO : 544 batches submitted to accumulate stats from 34816 documents (643572 virtual)
2021-07-19 12:41:11,149 : INFO : 545 batches submitted to accumulate stats from 34880 documents (644269 virtual)
2021-07-19 12:41:14,228 : INFO : 546 batches submitted to accumulate stats from 34944 documents (644724 virtual)
2021-07-19 12:41:14,830 : INFO : 547 batches submitted to accumulate stats from 35008 documents (645459 virtual)
2021-07-19 12:41:14,903 : INFO : 548 batches submitted to accumulate stats from 35072 documents (646017 virtual)
2021-07-19 12:41:17,708 : INFO : 550 batches submitted to accumulate stats from 35200 documents 

2021-07-19 12:42:29,500 : INFO : 628 batches submitted to accumulate stats from 40192 documents (702195 virtual)
2021-07-19 12:42:29,982 : INFO : 629 batches submitted to accumulate stats from 40256 documents (702201 virtual)
2021-07-19 12:42:30,711 : INFO : 630 batches submitted to accumulate stats from 40320 documents (702445 virtual)
2021-07-19 12:42:31,854 : INFO : 631 batches submitted to accumulate stats from 40384 documents (702509 virtual)
2021-07-19 12:42:32,370 : INFO : 632 batches submitted to accumulate stats from 40448 documents (703125 virtual)
2021-07-19 12:42:32,925 : INFO : 633 batches submitted to accumulate stats from 40512 documents (703400 virtual)
2021-07-19 12:42:35,328 : INFO : 636 batches submitted to accumulate stats from 40704 documents (703271 virtual)
2021-07-19 12:42:36,540 : INFO : 637 batches submitted to accumulate stats from 40768 documents (703835 virtual)
2021-07-19 12:42:37,548 : INFO : 638 batches submitted to accumulate stats from 40832 documents 

2021-07-19 12:43:54,008 : INFO : 728 batches submitted to accumulate stats from 46592 documents (752550 virtual)
2021-07-19 12:43:56,336 : INFO : 730 batches submitted to accumulate stats from 46720 documents (753081 virtual)
2021-07-19 12:43:56,650 : INFO : 731 batches submitted to accumulate stats from 46784 documents (754781 virtual)
2021-07-19 12:43:57,453 : INFO : 732 batches submitted to accumulate stats from 46848 documents (755753 virtual)
2021-07-19 12:43:58,845 : INFO : 733 batches submitted to accumulate stats from 46912 documents (756701 virtual)
2021-07-19 12:43:59,859 : INFO : 734 batches submitted to accumulate stats from 46976 documents (757416 virtual)
2021-07-19 12:44:00,011 : INFO : 735 batches submitted to accumulate stats from 47040 documents (757966 virtual)
2021-07-19 12:44:01,470 : INFO : 736 batches submitted to accumulate stats from 47104 documents (758806 virtual)
2021-07-19 12:44:03,059 : INFO : 737 batches submitted to accumulate stats from 47168 documents 

2021-07-19 12:45:24,768 : INFO : 836 batches submitted to accumulate stats from 53504 documents (798107 virtual)
2021-07-19 12:45:28,115 : INFO : 839 batches submitted to accumulate stats from 53696 documents (797613 virtual)
2021-07-19 12:45:29,233 : INFO : 840 batches submitted to accumulate stats from 53760 documents (800106 virtual)
2021-07-19 12:45:31,452 : INFO : 841 batches submitted to accumulate stats from 53824 documents (803253 virtual)


Save the preprocessed data, corpus and dictionary for later exploration  

In [ ]:
df.to_csv('data/data_for_interpretation.csv')

with open('models/corpus.pkl', 'wb') as f:
    pickle.dump(corpus, f)
    
dictionary.save('models/dictionary')